In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import math

# Fast Text on OtherContext
- year_of_publication 
- Publisher
- number_of_pages 
- awards

In [ ]:
books = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_cleaned_w2v.csv"
df_books_temp = pd.read_csv(books, sep = "|")
df_books = df_books_temp.copy()
df_books.head(1)

In [ ]:
#We keep only few columns to create the context of training
df1 = df_books[df_books["book_language"] == "en"]
- year_of_publication 
- Publisher 
- number_of_pages 
- awards
df1 = df1[["Publisher", "awards"]]
print(len(df_books))
print(len(df1))
df1[1000:1005]

# Concatenation of embeddings

### Desc_Bert_768 || AuthorContext_FastText_60

##### Data merge




In [4]:
df_bert_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|")
df_fastText_auth_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/authorsContext_fastText.csv", sep = "|")

In [5]:
df_bert = df_bert_temp.copy()
df_fastText_auth = df_fastText_auth_temp.copy()
print(len(df_bert))
print(len(df_fastText_auth))


105508
105508


In [6]:
df_bert.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN,0


In [7]:
#We only keep the embeddings
X1 = df_bert[["isbn", "book_language", "description_bert"]]
s1 = 768

X2 = df_fastText_auth["authorsContext_fastText"]
s2 = 60

df_embed = pd.concat([X1, X2], axis = 1)
print(len(df_embed))
df_embed.head()

105508


,isbn,book_language,description_bert,authorsContext_fastText
0,0439023483,en,0,"['-0.10757246970929005', '-0.09679322073085082..."
1,0316015849,en,"['-0.098250076', '-0.38007897', '0.8143984', '...","['-0.11777790052996527', '-0.08616064630088306..."
2,0525478817,en,"['-0.099239185', '-0.12767254', '0.6324886', '...","['-0.11273428555127353', '-0.08976304471012085..."
3,1416524797,en,"['-0.13360432', '-0.23354772', '0.47973692', '...","['-0.11270047592309614', '-0.08497616087940212..."
4,0062024035,en,"['-0.26884735', '-0.1411476', '0.30364788', '-...","['-0.13137996882200242', '-0.07718185376375913..."


In [8]:
#ratings of Book Crossing
#df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv", sep = ";", error_bad_lines=False, encoding="latin-1")
df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv", sep = "|", error_bad_lines=False, encoding="latin-1")
df_ratings = df_ratings_temp.copy()

In [9]:
#Merge between embeddings and ratings dataframe
df_embed = pd.merge(df_ratings, df_embed, left_on = "ISBN", right_on = "isbn")  
print(len(df_embed))
df_embed.head(1)

64148


,User-ID,ISBN,Book-Rating,isbn,book_language,description_bert,authorsContext_fastText
0,190708,044015393X,0.0,044015393X,en,0,"['-0.11191713949665427', '-0.08290330194202918..."


In [10]:
#Verification of not empty lines
lineOK = df_embed[ (df_embed["book_language"] == "en") & \
                   (df_embed["description_bert"] != '0') & \
                   (df_embed["authorsContext_fastText"] != '0')]
print(len(lineOK))

36013


##### Strings of embeddings transformed into float

In [11]:
#Transformation of a line of df_embed in a vector of float
#beacause bert embedding has been saved as a list of strings

MIN = min(df_embed["User-ID"])
MAX = max(df_embed["User-ID"])

def vectEmbed(l, s1, s2):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  b = np.zeros(s1 + s2 + 1)

  #1st copy of User-ID
  b[0] = (l['User-ID']-MIN)/(MAX-MIN)

  #Then handling of list of strings
  mi = -24.282314
  ma = 2.1074116
  v1 = l['description_bert'].split("'")
  b[1:(s1+1)] = [float(v1[i]) for i in range(1, len(v1), 2)]
  #b[1:(s1+1)] = [((float(v1[i]) - mi)/ (ma - mi)) for i in range(1, len(v1), 2)]

  mii = -0.6839812078202764
  maa = 0.6203103736042976
  v2 = l['authorsContext_fastText'].split("'")
  b[(s1+1):(s1 + s2 + 2)] = [float(v2[i]) for i in range(1, len(v2), 2)]
  #b[(s1+1):(s1 + s2 + 2)] = [((float(v2[i]) - mii)/ (maa - mii)) for i in range(1, len(v2), 2)]

  return b

In [ ]:
#print(df_embed["description_bert"][lineOK.index[0]])
#print("\n", df_embed["authorsContext_fastText"][lineOK.index[0]])
#print("\n", vectEmbed(df_embed.iloc[lineOK.index[0]], s1, s2))

In [12]:
#treatment of all lines of df_embed 
#The result will be a X_data matrix

def matEmbed(l, s1, s2, df):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  m = np.zeros((len(l), (s1 + s2 + 1)))
  
  ind = 0
  for i in l.index:

      m[ind] = vectEmbed(df.iloc[i], s1, s2)
      ind = ind + 1
  
  return m

In [13]:
#It takes about 1 min for 63 264 lines
mat_X_data = matEmbed(lineOK, s1, s2, df_embed)
mat_X_data.shape

(36013, 829)

##### Min / Max of embbedings values

In [ ]:
#First i performed the min on each lines corresponding to the book's description embedding
#=> so it remains a min for each column
print(mat_X_data[:, 1:(s1+1)].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, 1:(s1+1)].min(axis = 0))

(768,)


-24.282314

In [ ]:
#Same for the max
print(mat_X_data[:, 1:(s1+1)].max(axis = 0).shape)
max(mat_X_data[:, 1:(s1+1)].max(axis = 0))

(768,)


2.1074116

In [ ]:
#Same for the median
print(np.median(mat_X_data[:, 1:(s1+1)], axis = 0).shape)
np.median(np.median(mat_X_data[:, 1:(s1+1)], axis = 0))

(768,)


-0.0259018065

In [ ]:
#Same for the mean
print(np.mean(mat_X_data[:, 1:(s1+1)], axis = 0).shape)
np.mean(np.mean(mat_X_data[:, 1:(s1+1)], axis = 0))

(768,)


-0.039433989750688965

In [ ]:
nb_nb = mat_X_data[:, 1:(s1+1)].shape[0] * mat_X_data[:, 1:(s1+1)].shape[1]
r_m10 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] <= -10)).reshape(-1)) / (nb_nb), 2)
r_m10_m5 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > -10) & (mat_X_data[:, 1:(s1+1)] <= -5)).reshape(-1)) / (nb_nb), 2)
r_m5_m1 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > -5) & (mat_X_data[:, 1:(s1+1)] <= -1)).reshape(-1)) / (nb_nb), 2)
r_m1_0 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > -1) & (mat_X_data[:, 1:(s1+1)] <= 0)).reshape(-1)) / (nb_nb), 2)
r_0 = round(100 * np.sum(((mat_X_data[:, 1:(s1+1)] > 0)).reshape(-1)) / (nb_nb), 2)

print("Percentage of embeddings value under -10: \t\t", r_m10)
print("Percentage of embeddings value between -10 and -5: \t", r_m10_m5)
print("Percentage of embeddings value under -5 and -1: \t", r_m5_m1)
print("Percentage of embeddings value under -1 and 0: \t\t", r_m1_0)
print("Percentage of embeddings value above 0: \t\t", r_0)

Percentage of embeddings value under -10: 		 0.13
Percentage of embeddings value between -10 and -5: 	 0.0
Percentage of embeddings value under -5 and -1: 	 0.28
Percentage of embeddings value under -1 and 0: 		 52.1
Percentage of embeddings value above 0: 		 47.49


In [ ]:
#First i performed the min on each lines corresponding to the author context embedding
#=> so it remains a min for each column
print(mat_X_data[:, (s1+1):].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, (s1+1):].min(axis = 0))

(60,)


-0.6839812078202764

In [ ]:
#Same for the max
print(mat_X_data[:, (s1+1):].max(axis = 0).shape)
max(mat_X_data[:, (s1+1):].max(axis = 0))

(60,)


0.6203103736042976

### Desc_FastText_60 || AuthorContext_FastText_60

##### Data merge

In [22]:
df_fastText_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_fastText.csv", sep = "|")
df_fastText_auth_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/authorsContext_fastText.csv", sep = "|")

In [23]:
df_fastText = df_fastText_temp.copy()
df_fastText_auth = df_fastText_auth_temp.copy()
print(len(df_fastText))
print(len(df_fastText_auth))

105508
105508


In [24]:
df_fastText.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_fastFM
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN,0


In [25]:
#We only keep the embeddings
X1 = df_fastText[["isbn", "book_language", "description_fastFM"]]
s1 = 60

X2 = df_fastText_auth["authorsContext_fastText"]
s2 = 60

df_embed = pd.concat([X1, X2], axis = 1)
print(len(df_embed))
df_embed.head()

105508


,isbn,book_language,description_fastFM,authorsContext_fastText
0,0439023483,en,0,"['-0.10757246970929005', '-0.09679322073085082..."
1,0316015849,en,"['0.046032028402859235', '0.15718470524777384'...","['-0.11777790052996527', '-0.08616064630088306..."
2,0525478817,en,"['-0.015599380892056685', '0.34733651177241254...","['-0.11273428555127353', '-0.08976304471012085..."
3,1416524797,en,"['0.09067573049370284', '0.2065578283826736', ...","['-0.11270047592309614', '-0.08497616087940212..."
4,0062024035,en,"['-0.006060493551194668', '0.3125178421381861'...","['-0.13137996882200242', '-0.07718185376375913..."


In [26]:
#ratings of Book Crossing
#df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv", sep = ";", error_bad_lines=False, encoding="latin-1")
df_ratings_temp = pd.read_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv", sep = "|", error_bad_lines=False, encoding="latin-1")
df_ratings = df_ratings_temp.copy()

In [27]:
#Merge between embeddings and ratings dataframe
df_embed = pd.merge(df_ratings, df_embed, left_on = "ISBN", right_on = "isbn")  
print(len(df_embed))
df_embed.head(1)

64148


,User-ID,ISBN,Book-Rating,isbn,book_language,description_fastFM,authorsContext_fastText
0,190708,044015393X,0.0,044015393X,en,0,"['-0.11191713949665427', '-0.08290330194202918..."


In [28]:
#Verification of not empty lines
lineOK = df_embed[ (df_embed["book_language"] == "en") & \
                   (df_embed["description_fastFM"] != '0') & \
                   (df_embed["authorsContext_fastText"] != '0')]
print(len(lineOK))

36009


##### Strings of embeddings concatenated into float

In [29]:
#Transformation of a line of df_embed in a vector of float
#beacause bert embedding has been saved as a list of strings

MIN = min(df_embed["User-ID"])
MAX = max(df_embed["User-ID"])

def vectEmbed2(l, s1, s2):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  b = np.zeros(s1 + s2 + 1)

  #1st copy of User-ID
  b[0] = (l['User-ID']-MIN)/(MAX-MIN)

  #Then handling of list of strings
  mi = -1.8926484078168868
  ma = 2.0181634426116943
  v1 = l['description_fastFM'].split("'")
  b[1:(s1+1)] = [float(v1[i]) for i in range(1, len(v1), 2)]
  #b[1:(s1+1)] = [((float(v1[i]) - mi) / (ma - mi)) for i in range(1, len(v1), 2)]

  mii = -0.6839812078202764
  maa = 0.6203103736042976
  v2 = l['authorsContext_fastText'].split("'")
  b[(s1+1):(s1 + s2 + 2)] = [float(v2[i]) for i in range(1, len(v2), 2)]
  #b[(s1+1):(s1 + s2 + 2)] = [((float(v2[i]) - mii)/ (maa - mii)) for i in range(1, len(v2), 2)]

  return b

In [30]:
#treatment of all lines of df_embed 
#The result will be a X_data matrix

def matEmbed2(l, s1, s2, df):

  #for example bert embedding of s1 = 768 
  #          + fastText embedding of s2 = 60
  #          + User-ID
  m = np.zeros((len(l), (s1 + s2 + 1)))
  
  ind = 0
  for i in l.index:

      m[ind] = vectEmbed2(df.iloc[i], s1, s2)
      ind = ind + 1
  
  return m

In [31]:
#It takes about 1 min for 63 264 lines
mat_X_data = matEmbed2(lineOK, s1, s2, df_embed)
print(s1, s2, mat_X_data.shape)

60 60 (36009, 121)


##### Min / Max of embbedings values

In [ ]:
#First i performed the min on each lines corresponding to the book's description embedding
#=> so it remains a min for each column
print(mat_X_data[:, 1:(s1+1)].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, 1:(s1+1)].min(axis = 0))

(60,)


0.0

In [ ]:
#Same for the max
print(mat_X_data[:, 1:(s1+1)].max(axis = 0).shape)
max(mat_X_data[:, 1:(s1+1)].max(axis = 0))

(60,)


1.0

In [ ]:
#First i performed the min on each lines corresponding to the author context embedding
#=> so it remains a min for each column
print(mat_X_data[:, (s1+1):].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, (s1+1):].min(axis = 0))

(60,)


0.0

In [ ]:
#Same for the max
print(mat_X_data[:, (s1+1):].max(axis = 0).shape)
max(mat_X_data[:, (s1+1):].max(axis = 0))

(60,)


1.0

##### Strings of embeddings added as float

In [11]:
#Transformation of a line of df_embed in a vector of float
#beacause bert embedding has been saved as a list of strings

MIN = min(df_embed["User-ID"])
MAX = max(df_embed["User-ID"])

def vectEmbed3(l, s):

  #for example fastText embedding of s = 60 added with fastText embedding of s = 60
  #          + User-ID
  b_temp = np.zeros(s + 1)
  b = np.zeros(s + 1)

  #1st copy of User-ID
  b_temp[0] = (l['User-ID']-MIN)/(MAX-MIN)

  #Then handling of list of strings
  mi = -1.8926484078168868
  ma = 2.0181634426116943
  v1 = l['description_fastFM'].split("'")
  b_temp[1:] = [((float(v1[i]) - mi) / (ma - mi)) for i in range(1, len(v1), 2)]

  mii = -0.6839812078202764
  maa = 0.6203103736042976
  v2 = l['authorsContext_fastText'].split("'")
  b[1:] = [((float(v2[i]) - mii)/ (maa - mii)) for i in range(1, len(v2), 2)]

  #Real addition (and no more concatenation) of both embeddings
  b[1:] = b[1:] + b_temp[1:]

  return b

In [12]:
#treatment of all lines of df_embed 
#The result will be a X_data matrix

def matEmbed3(l, s, df):

  #for example fastText embedding of s = 60 added with fastText embedding of s = 60
  #          + User-ID
  m = np.zeros((len(l), (s + 1)))
  
  ind = 0
  for i in l.index:

      m[ind] = vectEmbed3(df.iloc[i], s)
      ind = ind + 1
  
  return m

In [13]:
#It takes about 1 min for 63 264 lines
mat_X_data = matEmbed3(lineOK, 60, df_embed)
print(mat_X_data.shape)

(63257, 61)


##### Min / Max of embbedings values

In [ ]:
#First i performed the min on each lines corresponding to the book's description embedding
#=> so it remains a min for each column
print(mat_X_data[:, 1:].min(axis = 0).shape)

#Then i take the min of this values vector
min(mat_X_data[:, 1:].min(axis = 0))

(60,)


0.5091274849896965

In [ ]:
#Same for the max
print(mat_X_data[:, 1:].max(axis = 0).shape)
max(mat_X_data[:, 1:].max(axis = 0))

(60,)


1.5535841137550046

### Selection of corresponding labels

In [32]:
Y_data = df_embed["Book-Rating"]
print(len(Y_data))
Y_data.head()

64148


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Book-Rating, dtype: float64

In [33]:
#treatment of all corresponding lines of Y_data
#=> same lines as the one used to create mat_X_data

def vectLabels(l, df):
  m = np.zeros((len(l), 1), dtype = int)
  
  ind = 0
  for i in l.index:
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [34]:
vect_Y_data = vectLabels(lineOK, Y_data)
vect_Y_data.shape

(36009, 1)

# Train / Test split

In [35]:
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [36]:
print("len(lineOK)", len(lineOK), "\n")

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

len(lineOK) 36009 

X_train.shape (28807, 121)
X_test.shape (7202, 121) 

y_train.shape (28807, 1)
y_test.shape (7202, 1)


# Ordinal regression

### Loading

In [19]:
!pip install mord
from mord import LogisticAT, OrdinalRidge

  Created wheel for mord: filename=mord-0.6-cp37-none-any.whl size=6007 sha256=af07a382947d1f0cd00cd68ff1c61e434c7864db6049ac9dc78e586e1448370d
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


### Training / Test

In [37]:
print(X_train[0:5].shape)
X_train[0:5]

(5, 121)


array([[ 2.04987699e-01,  1.47127885e-01,  1.82802604e-01,
        -4.70628025e-01, -8.84482123e-02, -1.67258042e-01,
        -2.95858964e-01, -2.10696034e-02,  3.41656124e-01,
        -2.52250425e-01, -1.11473936e-01, -6.98941290e-02,
        -2.85023838e-02,  1.72531436e-01,  3.90987873e-01,
        -2.62556293e-01, -1.30993329e-01, -5.65939009e-02,
         1.89685847e-01, -2.79178750e-01,  1.40381807e-01,
         2.48556793e-01, -9.21094717e-02,  2.08425504e-01,
        -9.08719271e-02,  2.03915347e-01,  1.56749922e-01,
         9.00656164e-02, -2.10771678e-01, -4.09622562e-01,
        -3.16563189e-02,  2.17218412e-01,  1.45367166e-01,
         2.14167310e-01, -3.98648638e-01,  7.53792249e-02,
        -9.46342766e-02, -2.15250719e-01,  6.25737458e-02,
        -2.49493346e-02,  1.35384908e-01,  2.68128222e-01,
        -2.97948254e-01,  1.08346593e-01, -2.18891302e-01,
         1.48785359e-01,  2.18800637e-01, -3.55247437e-02,
        -4.77795696e-01,  8.70218500e-02, -2.87509947e-0

In [38]:
Counter(np.squeeze(y_train))

Counter({0: 14242, 1: 14565})

In [39]:
#It takes about 35 min for desc_bert_768 || AuthorContext_FastText_60
LAT = LogisticAT(alpha=0., max_iter = 10000)
LAT.fit(X_train, np.squeeze(y_train))

ORD = OrdinalRidge(alpha=0., max_iter = 10000)
ORD.fit(X_train, np.squeeze(y_train))

OrdinalRidge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=10000,
             normalize=False, random_state=None, solver='auto', tol=0.001)

In [40]:
X_test[0:5]

array([[ 3.30612596e-02,  6.25395087e-03,  3.58330915e-01,
        -2.54287295e-01, -2.34830532e-01,  1.25054905e-01,
        -1.61436089e-01,  2.04381021e-01,  9.61399067e-02,
        -5.02320500e-02,  8.70689274e-02, -1.43338012e-02,
        -1.54947803e-01,  1.49978727e-01,  3.79466928e-01,
        -2.85250415e-01, -2.74131347e-01, -6.59154647e-02,
        -8.10925145e-02, -1.04431517e-01, -2.02628423e-01,
         1.43697540e-02, -9.89351272e-02,  3.54370800e-01,
         2.49538594e-02,  1.89138948e-01,  1.85053635e-02,
        -2.43961204e-01, -2.39516151e-02, -3.23334190e-01,
         2.14406310e-01,  2.61659481e-01, -2.21352990e-02,
         3.45975470e-02, -3.01087170e-01,  1.47312859e-01,
         7.14605107e-02,  1.18633122e-01, -9.20067503e-02,
         1.20049746e-01,  2.54116754e-01,  3.94422172e-01,
         1.08759427e-01,  3.49814376e-01, -1.11023669e-02,
         2.04694882e-01, -1.97863334e-01,  2.58036144e-01,
        -3.45397841e-01,  7.16784953e-02, -1.20531440e-0

In [41]:
Counter(np.squeeze(y_test))

Counter({0: 3561, 1: 3641})

In [42]:
y_pred_lat = LAT.predict(X_test)
y_pred_ord = ORD.predict(X_test)

### Unbalanced ratings

In [26]:
def ConfMat(y_pred, y_true):
  #misclassified elements for each classes
  compResTab = np.zeros(11)

  for i in range (len(y_test)):
    #The correct classe is y_test[i], so we increment its counter of misclassified if y_lin_pred[i] is not the same
    compResTab[int(y_test[i])] = compResTab[int(y_test[i])] + (y_pred[i] != y_test[i])
  
  p = Counter(np.squeeze(y_test))
  print("Class \tMisclassified \t    Percentage")
  for i in range(11):
     print("{0}\t{1}\t\t\t{2}".format(i, compResTab[i], round(100*(compResTab[i]/p[i]), 2)))
  print("\nTOTAL\t{0}\t\t\t{1}".format(compResTab.sum(), round(100*compResTab.sum()/len(y_test),2)))

In [27]:
print(Counter(y_pred_lat))

compRes_lat = [i for i in range(len(y_test)) if y_pred_lat[i] != y_test[i]]
print("Percentage of misclassified (LogisticAT): ", (len(compRes_lat) / len(y_test))*100)

ConfMat(y_pred_lat, y_test)

Counter({0: 12181, 5: 177, 4: 115, 6: 54, 3: 44, 2: 41, 1: 22, 7: 18})
Percentage of misclassified (LogisticAT):  39.51153967752134
Class 	Misclassified 	    Percentage
0	236.0			3.0
1	17.0			100.0
2	30.0			100.0
3	68.0			100.0
4	96.0			100.0
5	528.0			98.88
6	386.0			99.48
7	824.0			99.64
8	1160.0			100.0
9	719.0			100.0
10	935.0			100.0

TOTAL	4999.0			39.51


In [28]:
print(Counter(y_pred_ord))

compRes_ord = [i for i in range(len(y_test)) if y_pred_ord[i] != y_test[i]]
print("Percentage of misclassified (OrdinalRidge): ", (len(compRes_ord) / len(y_test))*100)

ConfMat(y_pred_ord, y_test)

Counter({3.0: 7930, 2.0: 2858, 4.0: 1698, 1.0: 108, 5.0: 58})
Percentage of misclassified (OrdinalRidge):  99.4783433449257
Class 	Misclassified 	    Percentage
0	7878.0			100.0
1	17.0			100.0
2	24.0			80.0
3	23.0			33.82
4	83.0			86.46
5	532.0			99.63
6	388.0			100.0
7	827.0			100.0
8	1160.0			100.0
9	719.0			100.0
10	935.0			100.0

TOTAL	12586.0			99.48


### Binary classes

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix_lat = confusion_matrix( y_pred_lat, y_test )
print(confusion_matrix_lat)
print("Confusion matrix (LogisticAT) :", 100*(confusion_matrix_lat/len(y_test)), sep = "\n")

print("Percentage of misclassified (LogisticAT): ", ((confusion_matrix_lat[0,1] + confusion_matrix_lat[1,0]) / len(y_test))*100)

[[1891 1466]
 [1670 2175]]
Confusion matrix (LogisticAT) :
[[26.25659539 20.35545682]
 [23.18800333 30.19994446]]
Percentage of misclassified (LogisticAT):  43.54346014995835


In [46]:
confusion_matrix_ord = confusion_matrix( y_pred_ord, y_test )
print(confusion_matrix_ord)
print("Confusion matrix (OrdinalRidge) :", 100*(confusion_matrix_ord/len(y_test)), sep = "\n")

print("Percentage of misclassified (OrdinalRidge): ", ((confusion_matrix_ord[0,1] + confusion_matrix_ord[1,0]) / len(y_test))*100)

[[1892 1460]
 [1669 2181]]
Confusion matrix (OrdinalRidge) :
[[26.27048042 20.27214663]
 [23.1741183  30.28325465]]
Percentage of misclassified (OrdinalRidge):  43.44626492640933


# Classical regression

In [64]:
from sklearn.linear_model import LinearRegression

### Unbalanced ratings

In [30]:
def MyRound(x): 
  return math.ceil(x) if math.ceil(x) - x < x - math.floor(x) else math.floor(x)

In [31]:
model_LinReg = LinearRegression()
model_LinReg.fit(X_train, np.squeeze(y_train))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [32]:
y_lin_pred = model_LinReg.predict(X_test)

In [33]:
y_lin_pred = list(map(MyRound, y_lin_pred))
print(Counter(y_lin_pred))

compRes_lin = [i for i in range(len(y_test)) if y_lin_pred[i] != y_test[i]]
print("Percentage of misclassified (LinearRegression): ", (len(compRes_lin) / len(y_test))*100)

ConfMat(y_lin_pred, y_test)

Counter({3: 7930, 2: 2858, 4: 1698, 1: 108, 5: 58})
Percentage of misclassified (LinearRegression):  99.4783433449257
Class 	Misclassified 	    Percentage
0	7878.0			100.0
1	17.0			100.0
2	24.0			80.0
3	23.0			33.82
4	83.0			86.46
5	532.0			99.63
6	388.0			100.0
7	827.0			100.0
8	1160.0			100.0
9	719.0			100.0
10	935.0			100.0

TOTAL	12586.0			99.48


# Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier

### Training / Test

In [48]:
#It takes about 3 min for desc_bert_768 || AuthorContext_FastText_60
modele_rf = RandomForestClassifier()
modele_rf.fit(X_train, np.squeeze(y_train))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
y_predict_rf = modele_rf.predict(X_test)

### Unbalanced ratings

In [37]:
print(Counter(y_predict_rf))

compRes_rf = [i for i in range(len(y_test)) if y_predict_rf[i] != y_test[i]]
print("Percentage of misclassified (RandomForestClassifier): ", (len(compRes_rf) / len(y_test))*100)

ConfMat(y_predict_rf, y_test)

Counter({0: 11507, 10: 315, 8: 287, 7: 183, 9: 140, 5: 104, 6: 80, 3: 15, 4: 12, 2: 5, 1: 4})
Percentage of misclassified (RandomForestClassifier):  41.46380018969333
Class 	Misclassified 	    Percentage
0	613.0			7.78
1	17.0			100.0
2	30.0			100.0
3	68.0			100.0
4	96.0			100.0
5	527.0			98.69
6	385.0			99.23
7	813.0			98.31
8	1123.0			96.81
9	707.0			98.33
10	867.0			92.73

TOTAL	5246.0			41.46


### Binary classes

In [50]:
confusion_matrix_rf = confusion_matrix( y_predict_rf, y_test)
print(confusion_matrix_rf)
print("Confusion matrix (RandomForestClassifier) :", 100*(confusion_matrix_rf/len(y_test)), sep = "\n")

print("Percentage of misclassified (RandomForestClassifier): ", ((confusion_matrix_rf[0,1] + confusion_matrix_rf[1,0]) / len(y_test))*100)

[[1799 1579]
 [1762 2062]]
Confusion matrix (RandomForestClassifier) :
[[24.97917245 21.92446543]
 [24.46542627 28.63093585]]
Percentage of misclassified (RandomForestClassifier):  46.3898916967509


# Logistic regression

In [51]:
from sklearn.linear_model import LogisticRegression

In [52]:
model_LogReg = LogisticRegression(max_iter = 10000, solver = "liblinear")

In [53]:
model_LogReg.fit(X_train, np.squeeze(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
y_predict_LogReg = model_LogReg.predict(X_test)

In [55]:
confusion_matrix_LogReg = confusion_matrix( y_predict_LogReg, y_test)
print(confusion_matrix_LogReg)
print("Confusion matrix (LogisticRegression) :", 100*(confusion_matrix_LogReg/len(y_test)), sep = "\n")

print("Percentage of misclassified (LogisticRegression): ", ((confusion_matrix_LogReg[0,1] + confusion_matrix_LogReg[1,0]) / len(y_test))*100)

[[1842 1445]
 [1719 2196]]
Confusion matrix (LogisticRegression) :
[[25.57622883 20.06387115]
 [23.8683699  30.49153013]]
Percentage of misclassified (LogisticRegression):  43.9322410441544
